In [1]:
import time
import requests
import json
from authlib.jose import JsonWebSignature
import pandas as pd
from dotenv import load_dotenv
import os

In [2]:
# Configuracion para que pandas muestre todas las columnas
pd.set_option("display.max_columns", None)

# Configuración para mostrar números en notación normal
pd.set_option("display.float_format", "{:.0f}".format)

In [3]:
load_dotenv()  # Carga las variables del archivo .env

token_id = os.getenv("API_TOKEN_ID")
token_secret = os.getenv("API_TOKEN_SECRET")

In [4]:
# Base URL and endpoint
base_url = "https://api.absolute.com"
endpoint = "/v3/reporting/devices"
page_size = 50  # Tamaño de página
# Inicializar lista para almacenar todos los datos
all_data = []

# Paginación: token para la siguiente página
next_page = None

while True:
    # Preparar el query string
    query_string = f"pageSize={page_size}"
    if next_page:
        query_string += f"&nextPage={next_page}"

    # Crear la solicitud
    request = {
        "method": "GET",
        "contentType": "application/json",
        "uri": endpoint,
        "queryString": query_string,
        "payload": {}
    }

    # Crear el payload y firmarlo con JsonWebSignature
    request_payload_data = {
        "data": request["payload"]
    }
    headers = {
        "alg": "HS256",
        "kid": token_id,
        "method": request["method"],
        "content-type": request["contentType"],
        "uri": request["uri"],
        "query-string": request["queryString"],
        "issuedAt": round(time.time() * 1000)
    }

    jws = JsonWebSignature()
    signed = jws.serialize_compact(headers, json.dumps(request_payload_data), token_secret)

    # Enviar la solicitud
    request_url = f"{base_url}/jws/validate"
    response = requests.post(request_url, signed, {"content-type": "text/plain"})

    # Verificar la respuesta
    if response.status_code != 200:
        print(f"Error: {response.status_code}, {response.text}")
        break

    # Procesar los resultados
    result = response.json()
    all_data.extend(result["data"])

    # Obtener el token para la siguiente página
    next_page = result.get("metadata", {}).get("pagination", {}).get("nextPage")

    # Terminar si no hay más páginas
    if not next_page:
        break

In [41]:
df_dispositivos = pd.DataFrame(all_data)

In [42]:
df_dispositivos.head(3)

,deviceUid,esn,accountUid,lastUpdatedDateTimeUtc,platformOSType,policyGroupName,src,firstCallDateTimeUtc,hdcStatus,geoStatus,espStatus,duStatus,agentStatus,fullSystemName,deviceName,systemManufacturer,systemModel,systemType,serialNumber,systemDirectory,bootDevice,locale,username,currentUsername,timeZone,totalPhysicalRamBytes,availablePhysicalRamBytes,totalVirtualMemoryBytes,availableVirtualMemoryBytes,pageFile,pageFileSpaceBytes,domain,agentVersion,ctesVersion,isCTESActive,localIp,publicIp,publicIpAddress,localIpAddress,unenrollmentDateTimeUtc,lastConnectedDateTimeUtc,battery,cpu,disks,displays,keyboards,memories,networkAdapters,operatingSystem,pointingDevices,printers,sounds,usbs,cameras,rrCountSummary,chassisType,rsvpStatus,volumes,sdcStatus,dlpStatus,bios,rnrSophosESC,avpInfo,espInfo,geoData,persistentAgentVersion,policyGroupUid,classification,deviceGroupIds,avgMinutesInUse,orgUnits,rnrHPTechPulse,cdRoms,isStolen,deviceFreezeActionStatus
0,f81ffad4-31cf-4504-8fe3-ecf2d4559be0,6LJCEBX5M9,fc39ea8e-61df-4330-aaca-a6ac74f0b025,2024-05-20T18:55:33.212Z,Unknown,Global Policy Group,upld,2024-05-20T18:30:04.067Z,"{'status': 'OK', 'featureType': 'HDC', 'lastDa...","{'isEnabled': True, 'featureType': 'GEO'}","{'isEnabled': True, 'featureType': 'ESP'}","{'isEnabled': True, 'featureType': 'DUR'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4c498db0-3ce3-4fd0-962c-4403474fe6ab,6LJCEBX5M9AA4I350001,fc39ea8e-61df-4330-aaca-a6ac74f0b025,2024-06-10T13:43:32.745Z,Windows,Global Policy Group,upld,2024-06-05T21:26:26.217Z,"{'status': 'OK', 'statusCode': 0, 'isEnabled':...","{'status': 'OK', 'statusCode': 0, 'isEnabled':...","{'status': 'OK', 'statusCode': 0, 'isEnabled':...","{'status': 'OK', 'statusCode': 0, 'isEnabled':...",D,LAP-QOR-CRP-03.aruntnet.local,LAP-QOR-CRP-03,Lenovo,THINKPAD T16 GEN 2,x64-based PC,PF4Q3GBK,C:\Windows\system32,\Device\HarddiskVolume1,Español (México),ARUNTNET\fruiz,ARUNTNET\fruiz,Hora verano Pacífico Sudamérica,16836866048,3261853696,37237960704,13753315328,C:\pagefile.sys,20401094656,aruntnet.local,8.994.0.9,1.0.0.3423,True,172.16.21.140,200.60.114.106,3359404650,2886735244,2024-06-06T00:03:44.695Z,2024-06-06T22:12:15.104Z,"{'id': '1724SMP5B10W51871', 'name': '5B10W5187...","{'id': 'CPU0', 'name': '13th Gen Intel(R) Core...","[{'id': '\\.\PHYSICALDRIVE0', 'name': '\\.\PHY...","[{'id': 'DesktopMonitor1', 'name': 'Monitor pr...","[{'id': 'HID\INTC816&COL01\3&10839A5C&0&0000',...","[{'id': 'Physical Memory 0', 'manufacturer': '...","[{'id': '0', 'name': 'Microsoft Kernel Debug N...","{'architecture': '64 bits', 'build': '19045', ...",[{'id': 'USB\VID_046D&PID_C077\5&2C15F40F&0&1'...,"[{'id': 'PDF24 Fax', 'name': 'PDF24 Fax', 'dri...",[{'id': 'INTELAUDIO\CTLR_DEV_51CA&LINKTYPE_02&...,[{'id': 'PCI\VEN_8086&DEV_51ED&SUBSYS_230E17AA...,[{'id': 'USB\VID_5986&PID_2142&MI_02\6&D7F3E0C...,{},10,{'isSupported': False},[{'id': '\\?\Volume{b941887a-b03c-4720-929e-61...,"{'featureType': 'SDC', 'calcStatus': 'NOT_INST...","{'isEnabled': False, 'featureType': 'DLP', 'ca...",{'id': 'LENOVO - 1440 N3QET44W (1.44 ) Lenovo ...,"{'trigger': 'Scheduled', 'version': '1.63.3', ...","{'antivirusName': 'Sophos Intercept X', 'antiv...",{'encryptionProductName': 'BitLocker Drive Enc...,"{'location': {'point': {'x': -77.0129919, 'y':...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,b0142679-a879-4808-850a-66fcb61fe825,6LJCEBX5M9AA4I350002,fc39ea8e-61df-4330-aaca-a6ac74f0b025,2024-12-17T13:32:35.471Z,Windows,Dispositivos sin Licencias,upld,2024-06-10T13:47:03.043Z,"{'status': 'OK', 'statusCode': 0, 'isEnabled':...","{'status': 'OK', 'statusCode': 0, 'isEnabled':...","{'status': 'OK', 'statusCode': 0, 'isEnabled':...","{'status': 'OK', 'statusCode': 0, 'isEnabled':...",A,PC-QOR-RENT-03.aruntnet.local,PC-QOR-RENT-03,Hewlett-Packard,HP PRO SFF 

In [43]:
df_dispositivos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123 entries, 0 to 122
Data columns (total 75 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   deviceUid                    123 non-null    object 
 1   esn                          123 non-null    object 
 2   accountUid                   123 non-null    object 
 3   lastUpdatedDateTimeUtc       123 non-null    object 
 4   platformOSType               123 non-null    object 
 5   policyGroupName              122 non-null    object 
 6   src                          123 non-null    object 
 7   firstCallDateTimeUtc         123 non-null    object 
 8   hdcStatus                    123 non-null    object 
 9   geoStatus                    123 non-null    object 
 10  espStatus                    123 non-null    object 
 11  duStatus                     123 non-null    object 
 12  agentStatus                  122 non-null    object 
 13  fullSystemName      

In [44]:
df_dispositivos= df_dispositivos[['deviceUid','serialNumber', 'systemManufacturer', 'systemModel', 
                                  'username', 'lastUpdatedDateTimeUtc', 'platformOSType', 'deviceName', 
                                  'totalPhysicalRamBytes', 'availablePhysicalRamBytes', 'domain', 'localIp',
                                  'cpu', 'disks', 'bios', 'avpInfo', 'geoData']]

In [45]:
df_dispositivos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123 entries, 0 to 122
Data columns (total 17 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   deviceUid                  123 non-null    object 
 1   serialNumber               122 non-null    object 
 2   systemManufacturer         121 non-null    object 
 3   systemModel                121 non-null    object 
 4   username                   121 non-null    object 
 5   lastUpdatedDateTimeUtc     123 non-null    object 
 6   platformOSType             123 non-null    object 
 7   deviceName                 121 non-null    object 
 8   totalPhysicalRamBytes      121 non-null    float64
 9   availablePhysicalRamBytes  121 non-null    float64
 10  domain                     121 non-null    object 
 11  localIp                    121 non-null    object 
 12  cpu                        121 non-null    object 
 13  disks                      121 non-null    object 

In [46]:
df_dispositivos["cpu_name"] = df_dispositivos["cpu"].apply(lambda x: x["name"] if isinstance(x, dict) else None)
df_dispositivos["bios_ver"] = df_dispositivos["bios"].apply(lambda x: x["version"] if isinstance(x, dict) else None)
df_dispositivos["latitude"] = df_dispositivos["geoData"].apply(
                                lambda x: x["location"]["point"]["y"] if isinstance(x, dict) else None)

df_dispositivos["longitude"] = df_dispositivos["geoData"].apply(
                                lambda x: x["location"]["point"]["x"] if isinstance(x, dict) else None)

df_dispositivos["antivirus"] = df_dispositivos["avpInfo"].apply(
                                lambda x: f"{x.get('antivirusName', 'Unknown')} {x.get('antivirusVersion', 'Unknown')}" 
                                if isinstance(x, dict) else None)

df_dispositivos['RAMTotal']= (df_dispositivos['totalPhysicalRamBytes'] / 1024) / 1024
df_dispositivos['RAMDisponible']= (df_dispositivos['availablePhysicalRamBytes'] / 1024) / 1024

df_dispositivos['DiskSpace'] = df_dispositivos['disks'].apply(
                                lambda x: next((item['sizeBytes'] for item in x if item['diskIndex'] == 0), None) 
                                if isinstance(x, list) and x else None)
df_dispositivos['DiskSpace'] = (df_dispositivos['DiskSpace'] / 1024) / 1024

In [47]:
# Eliminar columnas innecesarias
df_dispositivos= df_dispositivos.drop(['cpu','bios', 'geoData', 'avpInfo', 'totalPhysicalRamBytes', 'availablePhysicalRamBytes', 'disks'] , axis=1) 

# cambiar normbre de la columna
df_dispositivos= df_dispositivos.rename(columns={'cpu_name': 'cpu', 'bios_ver': 'bios'}) 


In [48]:
df_dispositivos.head(3)

,deviceUid,serialNumber,systemManufacturer,systemModel,username,lastUpdatedDateTimeUtc,platformOSType,deviceName,domain,localIp,cpu,bios,latitude,longitude,antivirus,RAMTotal,RAMDisponible,DiskSpace
0,f81ffad4-31cf-4504-8fe3-ecf2d4559be0,NaN,NaN,NaN,NaN,2024-05-20T18:55:33.212Z,Unknown,NaN,NaN,NaN,None,None,NaN,NaN,None,NaN,NaN,NaN
1,4c498db0-3ce3-4fd0-962c-4403474fe6ab,PF4Q3GBK,Lenovo,THINKPAD T16 GEN 2,ARUNTNET\fruiz,2024-06-10T13:43:32.745Z,Windows,LAP-QOR-CRP-03,aruntnet.local,172.16.21.140,13th Gen Intel(R) Core(TM) i7-1355U,LENOVO - 1440 N3QET44W (1.44 ) Lenovo - 1440,-12,-77,Sophos Intercept X 3.2.0.1560,16057,3111,976757
2,b0142679-a879-4808-850a-66fcb61fe825,4CE336D4QK,Hewlett-Packard,HP PRO SFF 400 G9 DESKTOP PC,ARUNTNET\monica.ulloa,2024-12-17T13:32:35.471Z,Windows,PC-QOR-RENT-03,aruntnet.local,172.16.65.18,12th Gen Intel(R) Core(TM) i7-12700,HPQOEM - 0 U03 Ver. 02.15.20,-12,-77,Sophos Endpoint Agent 2024.2.4.1.0,16054,3347,488382


In [49]:
print(df_dispositivos.iloc[3])
print(df_dispositivos.iloc[3]['bios'])

deviceUid                 6a5a0186-d33e-4441-b673-7746a9d6a05b
serialNumber                                        4CE336D5BV
systemManufacturer                             Hewlett-Packard
systemModel                       HP PRO SFF 400 G9 DESKTOP PC
username                             ARUNTNET\karen.valenzuela
lastUpdatedDateTimeUtc                2024-12-17T13:50:34.796Z
platformOSType                                         Windows
deviceName                                      PC-QOR-RENT-02
domain                                          aruntnet.local
localIp                                           172.16.65.56
cpu                        12th Gen Intel(R) Core(TM) i7-12700
bios                             HPQOEM - 0 U03 Ver. 02.15.20 
latitude                                                   -12
longitude                                                  -77
antivirus                   Sophos Endpoint Agent 2024.2.4.1.0
RAMTotal                                               

In [50]:
df_dispositivos.to_csv('data/abs_devices.csv', index=False)